# Train

In [4]:
# Convert to .py when done
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
import os

In [6]:
# Construct the path to move up two directory levels
higher_dir = os.path.abspath('../../')
os.chdir(higher_dir)

print(os.getcwd())

from module.config import *

C:\Users\MAMM01\OneDrive - ROC Mondriaan\Documenten\Workspace\CEDA\GitHub\Uitnodigingsregel\Uitnodigingsregel


In [8]:
# Define file paths
user_data_train_path = 'data/raw/user_data/train.csv'
user_data_pred_path = 'data/raw/user_data/pred.csv'
synth_train_path = 'data/raw/synth_data_train.csv'
synth_pred_path = 'data/raw/synth_data_pred.csv'


# Check if train.csv and pred.csv exist in user_data folder
if os.path.exists(user_data_train_path) and os.path.exists(user_data_pred_path):
    train_df = pd.read_csv(user_data_train_path, sep = '\t')
    pred_df = pd.read_csv(user_data_pred_path, sep = '\t')
else:
    train_df = pd.read_csv(synth_train_path, sep = '\t')
    pred_df = pd.read_csv(synth_pred_path, sep = '\t')

In [10]:
### Add verbose for progress bar 
# Random Forest Regressor model
def randomforestregressormodel_train(dataset_train, random_state = RANDOM_SEED):
    X = dataset_train.drop("Dropout", axis=1).values
    y = dataset_train.Dropout.values

  
    rf = RandomForestRegressor(random_state=random_state)
    parameters = {
    'bootstrap': [True, False],
    'max_depth': [2, 3, 4],
    'max_features': [3, 4, 5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2, 3, 5],
    'n_estimators': [100, 200, 300]}

    grid_model = GridSearchCV(rf, parameters, n_jobs = -1)
    grid_model.fit(X, y) 
    best_params = grid_model.best_params_
    best_rf_model = RandomForestRegressor(**best_params)
    best_rf_model.fit(X, y) 
    return best_rf_model


In [12]:
### Add LASSO and SVM

In [27]:
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler


## Use MinMax scaler to ensure all values are between 0 and 1 and prevent model bias. Alternatives are RobustScaler (based on percentiles so 
## immune to large outliers) or StandardScaler (if the data has a normal distribution) 

train_df_scaled = MinMaxScaler().fit_transform(train_df)


In [29]:
# Maak een functie aan voor de conventionele evaluatie van de voorspelling
def conventionele_evaluatie(model="lasso", data = train_df_scaled):
    # Bepaal de evaluatie categorie
    data["True Positive {}".format(model)] = np.where(((data["yhat2"] >= 0.5) & (data["y_test"] == 1)), 1, 0)
    data["False Positive {}".format(model)] = np.where(((data["yhat2"] >= 0.5) & (data["y_test"] == 0)), 1, 0)
    data["True Negative {}".format(model)] = np.where(((data["yhat2"] <= 0.5) & (data["y_test"] == 0)), 1, 0)
    data["False Negative {}".format(model)] = np.where(((data["yhat2"] <= 0.5) & (data["y_test"] == 1)), 1, 0)

    conv_precision = data["True Positive {}".format(model)].sum()/ (data["True Positive {}".format(model)].sum() + data["False Positive {}".format(model)].sum())
    conv_sensitivity = data["True Positive {}".format(model)].sum()/ (data["True Positive {}".format(model)].sum() + data["False Negative {}".format(model)].sum())
    # Maak een simpele confusionmatrix
    total_dict = {"Actual 0": [data["True Negative {}".format(model)].sum(), data["False Positive {}".format(model)].sum()],
                  "Actual 1": [data["False Negative {}".format(model)].sum(), data["True Positive {}".format(model)].sum()]}
    total = pd.DataFrame(data=total_dict, index = ["Predicted 0", "Predicted 1"])

    return total, conv_precision, conv_sensitivity

In [ ]:
# Lasso model after changes 
def lassomodel(train_df_scaled):
    X = train_df_scaled.drop("Dropout", axis=1).values
    y = train_df_scaled.Dropout.values
    ## Use LassoCV (cross-validation) to find the best alpha for LASSO model
    lasso_cv_model = LassoCV(cv=5, random_state=RANDOM_SEED)
    lasso_cv_model.fit(X_train, y_train)

  
    return yhat2_data, lasso_coefficients

## Run model
yhat2_data_lasso, lasso_coefficients = lassomodel() 
#yhat2_data_lasso = dynamische_evaluatie(model="lasso", data = yhat2_data_lasso)
total_lasso, conv_precision_lasso, conv_sensitivity_lasso = conventionele_evaluatie(model="lasso", data = yhat2_data_lasso)
yhat2_data_lasso['precisionlasso'] = yhat2_data_lasso['precisionlasso'].apply(lambda x: x*100)
yhat2_data_lasso['recalllasso'] = yhat2_data_lasso['recalllasso'].apply(lambda x: x*100)

In [ ]:
  ## Predict 
    yhat = model.predict(X_train)   
    yhat2 = model.predict(X_test)


    ## Sort results
    yhat2_data["yhat2_rank"] = yhat2_data["yhat2"].rank(method = 'dense', ascending=False)
    yhat2_data = yhat2_data.sort_values(by=["yhat2"], ascending=False).reset_index(drop=True)
    yhat2_data["i"] = yhat2_data.index + 1

    ## Add column with binary output of yhat2 based on mean 
    mean_lasso = yhat2_data.loc[:, 'yhat2'].mean()
    df_total = pd.DataFrame(data = yhat2_data['yhat2'])
    df_total.rename(columns={'yhat2': 'binary_output'}, inplace=True)
    df_total1 = df_total.applymap(lambda x: 1 if x > mean_lasso else 0)
    yhat2_data = pd.concat([yhat2_data, df_total1], axis=1)
    
    ## Features to column names 
    feature_names_after_preprocessing = dataset.drop("Dropout", axis=1).columns.tolist()  # Mismatch after use of SMOTE because of increased dimensions of X_train
    lasso_coefficients = get_coefficient_table(X_train, model)  # extra
    lasso_coefficients.index = feature_names_after_preprocessing# extra
